<a href="https://colab.research.google.com/github/hoangducsn00/webScrape_tiki/blob/main/scrapingweb_tikicomment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content/drive/My Drive/devC/clone_data
!ls

/content/drive/My Drive/devC/clone_data
 crawl-tiki-products  ' SCRAPE DYNAMIC websites.ipynb'
 data		       scrapingweb_tikicomment.ipynb
 data_comment_tiki


In [3]:
#import library
from bs4 import BeautifulSoup
import requests
import json
import csv
import pandas as pd
import urllib, json

In [4]:
names = ['may+tinh', 'sach+tieng+anh']

In [5]:
name = names[1]
name

'sach+tieng+anh'

In [6]:

laptop_page_url = "https://tiki.vn/search?q=" + name
product_url = "https://tiki.vn/search?q=" + name

product_id_file = "/content/drive/My Drive/devC/clone_data/data_comment_tiki/" + name + ".txt"

def crawl_product_id():
    product_list = []
    i = 1
    while (True):
        print("Crawl page: ", i)
        response = requests.get(laptop_page_url.format(i))
        parser = BeautifulSoup(response.text, 'html.parser')

        product_box = parser.findAll(class_="product-item")

        if (len(product_box) == 0):
            break

        for product in product_box:
            product_list.append(product.get("data-id"))

        i += 1
        if (i == 1000):
          break


    return product_list, i

def save_product_id(product_list=[]):
    file = open(product_id_file, "w+")
    str = "\n".join(product_list)
    file.write(str)
    file.close()
    print("Save file: ", product_id_file)

def crawl_product(product_list=[]):
    product_detail_list = []
    for product_id in product_list:
        response = requests.get(product_url.format(product_id))
        if (response.status_code == 200):
            product_detail_list.append(response.text)
            print("Crawl product: ", product_id, ": ", response.status_code)
    return product_detail_list



In [6]:
# crawl product id
product_list = []
product_list, page = crawl_product_id()

print("No. Page: ", page)
print("No. Product ID: ", len(product_list))

Crawl page:  1
Crawl page:  2
Crawl page:  3
Crawl page:  4
Crawl page:  5
Crawl page:  6
Crawl page:  7
Crawl page:  8
Crawl page:  9
Crawl page:  10
Crawl page:  11
Crawl page:  12
Crawl page:  13
Crawl page:  14
Crawl page:  15
Crawl page:  16
Crawl page:  17
Crawl page:  18
Crawl page:  19
Crawl page:  20
Crawl page:  21
Crawl page:  22
Crawl page:  23
Crawl page:  24
Crawl page:  25
Crawl page:  26
Crawl page:  27
Crawl page:  28
Crawl page:  29
Crawl page:  30
Crawl page:  31
Crawl page:  32
Crawl page:  33
Crawl page:  34
Crawl page:  35
Crawl page:  36
Crawl page:  37
Crawl page:  38
Crawl page:  39
Crawl page:  40
Crawl page:  41
Crawl page:  42
Crawl page:  43
Crawl page:  44
Crawl page:  45
Crawl page:  46
Crawl page:  47
Crawl page:  48
Crawl page:  49
Crawl page:  50
Crawl page:  51
Crawl page:  52
Crawl page:  53
Crawl page:  54
Crawl page:  55
Crawl page:  56
Crawl page:  57
Crawl page:  58
Crawl page:  59
Crawl page:  60
Crawl page:  61
Crawl page:  62
Crawl page:  63
C

In [ ]:
# save product id for backup
save_product_id(product_list)

In [8]:
df = pd.read_csv(product_id_file, header=None)
df

,0
0,45438055
1,25494701
2,54433518
3,54433520
4,25073118
...,...
47947,25979234
47948,5743619
47949,476963
47950,3840811


In [9]:
df.shape

(47952, 1)

In [30]:
separate_df = []

In [31]:
separate_df.append(df.iloc[40000 : 40500])
separate_df.append(df.iloc[40500 : 42000])

In [ ]:
for i in range(12,23):
  print(i)

In [32]:

for i in range(21,23):
  df_x = df.iloc[i*2000 : (i+1)*2000]
  separate_df.append(df_x)


In [33]:
separate_df.append(df.iloc[46000 : ])

In [34]:
len(separate_df)

5

In [20]:
list_data = []
def get_dict_from_id_list(df):
  for id in df[0]:
    url = 'https://tiki.vn/api/v2/reviews?product_id=' + str(id) + "&sort=score%7Cdesc,id%7Cdesc,stars%7Call&page=1&limit=10&include=comments"
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    list_data.append(data)
  return list_data

In [35]:
list_data_loop = []
num = 22
for i in separate_df:
  list_data_loop = get_dict_from_id_list(i)
  contents = []
  titles = []
  ratings = []
  for data in list_data_loop:
    for dict_ in data['data']:
      for key, value in dict_.items():
        if key == 'content':
          contents.append(value)
        if key == 'rating':
          ratings.append(value)
        if key == 'title':
          titles.append(value)
  data_save = pd.DataFrame({'ratings': ratings, 'titles': titles, 'comments': contents})
  data_frame_notna = data_save.dropna()
  path = '/content/drive/My Drive/devC/clone_data/data_comment_tiki/' + name + str(num) + '.csv'
  data_frame_notna.to_csv(path, index=False)
  print("Saved ", num)
  num = num + 1

Saved  22
Saved  23
Saved  24
Saved  25
Saved  26


In [24]:
#save data as csv file
path = '/content/drive/My Drive/devC/clone_data/data_comment_tiki/' + name + '1' + '.csv'
data_frame_notna.to_csv(path, index=False)